Testing the celeb recognition package

https://pypi.org/project/celeb-detector/

In [8]:
# install necessary libraries:

import celeb_detector 
img_path = 'sample_image.jpg' 
celeb_detector.celeb_recognition(img_path)

AttributeError: module 'celeb_detector' has no attribute 'celeb_recognition'

Ditching this & trying something else:

https://www.kaggle.com/code/vinayakshanawad/celebrity-face-recognition-vggface-model/notebook


In [6]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import precision_recall_curve,accuracy_score,f1_score,precision_score,recall_score

In [9]:
import os
import numpy as np

class IdentityMetadata:
    def __init__(self, base, name, file):
        self.base = base
        self.name = name  # identity name
        self.file = file  # image file name

    def __repr__(self):
        return self.image_path()

    def image_path(self):
        return os.path.join(self.base, self.name, self.file) 
    
def load_metadata(path):
    metadata = []
    for i in os.listdir(path):
        for f in os.listdir(os.path.join(path, i)):
            ext = os.path.splitext(f)[1]
            if ext.lower() in ['.jpg', '.jpeg']:
                metadata.append(IdentityMetadata(path, i, f))
    return np.array(metadata)

# Define the dataset path
source_dir = os.path.join('/kaggle', 'input', 'pins-face-recognition', '105_classes_pins_dataset')

# Load metadata
metadata = load_metadata(source_dir)
print(f"Loaded metadata for {len(metadata)} images.")

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/pins-face-recognition/105_classes_pins_dataset'

In [13]:
import os
import requests
from bs4 import BeautifulSoup
import shutil
import time

def create_folder(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

def get_celebrity_image_urls(celeb_name):
    search_url = f"https://www.imdb.com/find?q={celeb_name.replace(' ', '+')}&s=nm"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    response = requests.get(search_url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve search results for {celeb_name}. Status code: {response.status_code}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the first result's link to the celebrity profile
    result = soup.find('td', class_='result_text')
    
    if result and result.a:
        celeb_link = "https://www.imdb.com" + result.a['href']
        print(f"Found profile for {celeb_name}: {celeb_link}")
    else:
        print(f"No IMDb profile found for {celeb_name}.")
        return []

    celeb_response = requests.get(celeb_link, headers=headers)

    if celeb_response.status_code != 200:
        print(f"Failed to retrieve profile for {celeb_name}. Status code: {celeb_response.status_code}")
        return []

    celeb_soup = BeautifulSoup(celeb_response.text, 'html.parser')

    # Fetch images from the profile page
    image_urls = []
    
    img_tags = celeb_soup.find_all('img')
    for img_tag in img_tags:
        img_url = img_tag.get('src')
        if img_url and 'media' in img_url and img_url not in image_urls:
            image_urls.append(img_url)
    
    # Limit to a few images to avoid excess requests
    return image_urls[:5]

def download_images(celeb_name, image_urls):
    celeb_folder = os.path.join('celebrity_images', celeb_name)
    create_folder(celeb_folder)
    for idx, img_url in enumerate(image_urls):
        try:
            img_response = requests.get(img_url, stream=True)
            img_filename = os.path.join(celeb_folder, f"{celeb_name.replace(' ', '_')}_{idx+1}.jpg")
            if img_response.status_code == 200:
                with open(img_filename, 'wb') as out_file:
                    shutil.copyfileobj(img_response.raw, out_file)
                print(f"Downloaded {img_filename}")
            else:
                print(f"Failed to download image for {celeb_name} from {img_url}. Status code: {img_response.status_code}")
        except Exception as e:
            print(f"Error downloading image {img_url}: {e}")

# Example usage
celebs = ["Tom Hanks", "Scarlett Johansson", "Will Smith"]  # List of celebrities to scrape images for
create_folder("celebrity_images")

for celeb in celebs:
    print(f"Fetching images for {celeb}...")
    urls = get_celebrity_image_urls(celeb)
    if urls:
        download_images(celeb, urls)
    time.sleep(1)  # Sleep between requests


Fetching images for Tom Hanks...
No IMDb profile found for Tom Hanks.
Fetching images for Scarlett Johansson...
No IMDb profile found for Scarlett Johansson.
Fetching images for Will Smith...
No IMDb profile found for Will Smith.


In [5]:
import face_recognition
import os
from PIL import Image, ImageDraw, ImageFont

# Load known celebrity faces from the "celebrities" folder
def load_celebrities(celebrities_folder="celebrities"):
    known_face_encodings = []
    known_face_names = []
    
    for filename in os.listdir(celebrities_folder):
        if filename.endswith((".jpg", ".png")):
            path = os.path.join(celebrities_folder, filename)
            # Load the celebrity image
            image = face_recognition.load_image_file(path)
            # Get the face encoding for the image
            encoding = face_recognition.face_encodings(image)
            if encoding:  # Make sure there's a face detected
                known_face_encodings.append(encoding[0])
                # Get the name from the filename (without extension)
                known_face_names.append(os.path.splitext(filename)[0])
    return known_face_encodings, known_face_names

# Function to recognize celebrities in a new image
def recognize_celebrities(input_image_path, known_face_encodings, known_face_names):
    # Load the input image
    image = face_recognition.load_image_file(input_image_path)
    face_locations = face_recognition.face_locations(image)
    face_encodings = face_recognition.face_encodings(image, face_locations)
    
    # Convert image to PIL format for drawing (optional)
    pil_image = Image.fromarray(image)
    draw = ImageDraw.Draw(pil_image)

    # Optionally load a font
    font = ImageFont.load_default()

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # See if the face is a match for any known celebrity
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        
        name = "Unknown"
        if True in matches:
            # Find the closest match
            match_index = matches.index(True)
            name = known_face_names[match_index]
            # Print the name of the recognized celebrity
            print(f"Recognized: {name}")
        
        # Draw a box around the face
        draw.rectangle(((left, top), (right, bottom)), outline=(0, 255, 0), width=2)
        
        # Calculate text size using textbbox and add a label below the face
        text_size = draw.textbbox((0, 0), name, font=font)
        text_width, text_height = text_size[2] - text_size[0], text_size[3] - text_size[1]
        
        # Draw a label with the name below the face
        draw.rectangle(((left, bottom), (right, bottom + text_height + 10)), fill=(0, 255, 0), outline=(0, 255, 0))
        draw.text((left + 6, bottom + 5), name, fill=(255, 255, 255), font=font)
    
    # Save the labeled image to a file
    output_path = "output_image.jpg"
    pil_image.save(output_path)
    print(f"Image saved as {output_path}")

# Main program
if __name__ == "__main__":
    # Load the known celebrities
    known_face_encodings, known_face_names = load_celebrities()

    # Path to the input image you want to recognize
    input_image_path = "/workspace/DS4002Project3/celebrities/Margot-Robbie/Margot-Robbie-1.png"  # Change this to your test image path

    # Recognize celebrities
    recognize_celebrities(input_image_path, known_face_encodings, known_face_names)

Image saved as output_image.jpg
